In [39]:


from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import math
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import select

from datetime import datetime, timedelta,date
from typing import Union

In [40]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [41]:


def getrowdata(row):
    try:
        title = row.find('span', class_='a-size-medium a-color-base a-text-normal').text
    except:
        title = None
    try:
        author = row.find('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style').text
    except:
        author = None
    try:
        publishDate =  row.find('span', class_='a-size-base a-color-secondary a-text-normal').text
    except:
        publishDate = None
    try:
        mprice = row.find('div', class_ = "puisg-col-inner").find(attrs = {'data-cy' : "price-recipe"}).find(class_ = 'a-offscreen').text
    except:
        mprice = None
    try:
        morgprice = row.find('div', class_ = "puisg-col-inner").find(attrs = {'data-cy' : "price-recipe"}).find(class_="a-price a-text-price").find(class_="a-offscreen").text
    except:
        morgprice = None
    try:
        sprice = row.find('div', class_ = "a-row a-spacing-mini").find_all('div', class_ = 'a-row a-size-base a-color-base')[1].find('span', class_ ='a-offscreen').text
    except:
        sprice = None
    try:
        sorgprice = row.find('div', class_="a-row a-spacing-mini").find_all('div', class_="a-row a-size-base a-color-base")[1].find('span', class_="a-offscreen").text
    except:
        sorgprice = None
    try:
        mVer = row.find('div', class_ = "puisg-col-inner").find(attrs = {'data-cy' : "price-recipe"}).find(class_ = 'a-row a-spacing-mini a-size-base a-color-base').text
    except:
        mVer = None
    try:
        sVer = row.find('div', class_ = "a-row a-spacing-mini").find_all('div', class_ = 'a-row a-size-base a-color-base')[0].text
    except:
        sVer = None
    
    output = {
    'title': title,
    'author': author,
    'publishDate': publishDate,
    'mainVersion':mVer,
    'mainPrice':mprice,
    'mainOriginalPrice':morgprice,
    'subVersion':sVer,
    'subPrice':sprice,
    'subOriginalPrice':sorgprice,
        
    }
    
    return output


In [42]:
def timeparser(dtime:str)-> Union[date, None]:
    month_dict = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12
    }
    try:
        month, day, year = dtime.split()
        day = int(day.replace(',',''))
        month = month_dict[month.replace(' ','')]
        year = int(year.replace(' ',''))
        return date(year,month,day)
    except:
        return None


def Findsermin(series:pd.Series)-> date:
    min = date.today()
    for i in series:
        try:
            if i != None and i<min:
                min = i
        except:
            print(i,min)
            pass
    return min
        
        
    

In [43]:
from selenium.common import NoSuchElementException

#initilize





#driver = webdriver.Chrome( options=chrome_options)
#driver.get("https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45=8&p_46=before&p_47=2024&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")
#driver.add_cookie({'name' : 'csm-hit','value':'tb:s-R614YZC0MNEE8MA4GS26|1723461833540&t:1723461833540&adb:adblk_no'})
#driver.add_cookie({'name' : 'lc-main','value':'en_US'})
#
#try:
#    button = driver.find_element(By.ID, 'nav-global-location-popover-link')
#    button.click()
#    time.sleep(3)
#    input = '10001'
#    search = driver.find_element(By.ID, "GLUXZipUpdateInput")
#    search.send_keys(input)
#    time.sleep(3)
#    button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
#    button.click()
#    time.sleep(3)
#    button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
#    button[1].click()
#    time.sleep(3)
#except IndexError:
#    pass
#pagelen = driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-disabled"]').text
#soup = BeautifulSoup(driver.page_source, 'html.parser')
#rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
#for row in rows:
#    data.append(getrowdata(row))
#time.sleep(5)
df = pd.read_csv('./harpercollins_date_data.csv')
df['publishDate_'] = df['publishDate'].apply(timeparser)
stucked = 0

while Findsermin(df['publishDate_']) > date.fromisoformat('1600-01-01'):
    data = []
    reqdate = Findsermin(df['publishDate_'])
    driver = webdriver.Chrome( options=chrome_options)
    #driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=41&Adv-Srch-Books-Submit.y=16&p_45={reqdate.month}&p_46=before&p_47={reqdate.year}&qid=1723495742&unfiltered=1&ref=sr_pg_2")
    driver.get(f"https://www.amazon.com/advanced-search/books")
    driver.find_element(By.NAME,'field-publisher').send_keys('harpercollins')
    sel = driver.find_element(By.NAME,'field-p_n_condition-type')
    sel = select.Select(sel)
    sel.select_by_index(1)
    time.sleep(3)
    sel = driver.find_element(By.NAME,'sort')
    sel = select.Select(sel)
    sel.select_by_value("daterank")
    time.sleep(6)
    sel = driver.find_element(By.NAME,'p_46')
    sel = select.Select(sel)
    sel.select_by_index(2)
    time.sleep(3)
    sel = driver.find_element(By.NAME,'p_45')
    sel = select.Select(sel)
    sel.select_by_index(reqdate.month-1)
    time.sleep(3)
    driver.find_element(By.NAME,'p_47').send_keys(reqdate.year)
    driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
    time.sleep(2)
    
    while BeautifulSoup(driver.page_source, 'html.parser').title.text == 'Sorry! Something went wrong!':
        driver.refresh()
        if BeautifulSoup(driver.page_source, 'html.parser').title.text != 'Sorry! Something went wrong!':
            break
        stucked += 1
        driver.quit()
        time.sleep(stucked*50)
        driver.get(f"https://www.amazon.com/advanced-search/books")
        driver.find_element(By.NAME,'field-publisher').send_keys('harpercollins')
        sel = driver.find_element(By.NAME,'field-p_n_condition-type')
        sel = select.Select(sel)
        sel.select_by_index(1)
        time.sleep(3)
        sel = driver.find_element(By.NAME,'sort')
        sel = select.Select(sel)
        sel.select_by_value("daterank")
        time.sleep(6)
        sel = driver.find_element(By.NAME,'p_46')
        sel = select.Select(sel)
        sel.select_by_index(2)
        time.sleep(3)
        sel = driver.find_element(By.NAME,'p_45')
        sel = select.Select(sel)
        sel.select_by_index(reqdate.month-1)
        time.sleep(3)
        driver.find_element(By.NAME,'p_47').send_keys(reqdate.year)
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
        time.sleep(3)
        driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
    try:
        button = driver.find_element(By.ID, 'nav-global-location-popover-link')
        button.click()
        time.sleep(3)
        input = '10001'
        search = driver.find_element(By.ID, "GLUXZipUpdateInput")
        search.send_keys(input)
        time.sleep(3)
        button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
        button.click()
        time.sleep(3)
        button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
        button[1].click()
        time.sleep(3)
    except IndexError:
        print('indexerror')
        pass
    except:
        pass
    
    try:
        pagelen = driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-disabled"]').text
        pagelen = int(pagelen)
    except:
        pagelen=1
        
    
    
    for i in range(pagelen-1):
        #driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins&s=daterank&Adv-Srch-Books-Submit.x=41&Adv-Srch-Books-Submit.y=16&p_45={reqdate.month}&p_46=before&p_47={reqdate.year}&unfiltered=1&ref=sr_adv_b&page={i+1}")
        if random.randint(0,1) == 1:
            driver.refresh()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        while soup.title.text == 'Sorry! Something went wrong!':
            driver.refresh()
            if soup.title.text != 'Sorry! Something went wrong!':
                break
            stucked += 1
            time.sleep(stucked*60 if stucked<5 else 180)
            print(stucked)
            driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i+2}&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45={reqdate.month-1}&p_46=before&p_47={reqdate.year}&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            if stucked > 5:
                driver.quit()
                driver = webdriver.Chrome( options=chrome_options)
                driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i+2}&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45={reqdate.month-1}&p_46=before&p_47={reqdate.year}&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")
                try:
                    button = driver.find_element(By.ID, 'nav-global-location-popover-link')
                    button.click()
                    time.sleep(3)
                    input = '10001'
                    search = driver.find_element(By.ID, "GLUXZipUpdateInput")
                    search.send_keys(input)
                    time.sleep(3)
                    button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
                    button.click()
                    time.sleep(3)
                    button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
                    button[1].click()
                    time.sleep(3)
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
                    time.sleep(3)
                    driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
                except:
                    pass
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(3)
        rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
        for row in rows:
            data.append(getrowdata(row))
        time.sleep(random.randint(6,10))
        if driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text != str(pagelen):
            try:
                driver.find_element(By.CSS_SELECTOR,"a[class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'").click()
            except NoSuchElementException:
                pass
        elif driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text == str(pagelen):
            break
    dftemp = pd.DataFrame(data)
    print(data[1:30])
    print(dftemp.head)
    dftemp['publishDate_'] = dftemp['publishDate'].apply(timeparser)
    df = pd.concat([df, dftemp])
    
    
        
    df.to_csv(f"./data/harpercollins_{Findsermin(df['publishDate_']).strftime('%Y_%m_%d')}.csv", index=False)
    df.to_csv("./harpercollins_date_data.csv")

1
2
3
[{'title': 'Wander Lost', 'author': 'Laura Martin', 'publishDate': 'Jun 25, 2024', 'mainVersion': 'Hardcover ', 'mainPrice': '$19.99', 'mainOriginalPrice': None, 'subVersion': 'Audible Audiobook ', 'subPrice': '$0.00', 'subOriginalPrice': '$0.00'}, {'title': 'Scare Walk', 'author': 'Andaleeb Wajid', 'publishDate': 'Jun 24, 2024', 'mainVersion': 'Kindle ', 'mainPrice': '$12.99', 'mainOriginalPrice': None, 'subVersion': 'Paperback ', 'subPrice': '$28.95', 'subOriginalPrice': '$28.95'}, {'title': 'Why Mummy Drinks on Holiday: The hilarious new beach read from the bestselling author of Why Mummy Drinks', 'author': 'Gill Sims', 'publishDate': 'Jun 20, 2024', 'mainVersion': 'Kindle ', 'mainPrice': '$12.99', 'mainOriginalPrice': None, 'subVersion': 'Audible Audiobook ', 'subPrice': '$0.00', 'subOriginalPrice': '$0.00'}, {'title': "Little Critter: Lucky Ducky's Very First Day of School (My First I Can Read)", 'author': 'Mercer Mayer', 'publishDate': 'Jun 18, 2024', 'mainVersion': 'Paperb

AttributeError: 'DataFrame' object has no attribute 'publishDate'

In [58]:
from selenium.common import NoSuchElementException

#initilize





#driver = webdriver.Chrome( options=chrome_options)
#driver.get("https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45=8&p_46=before&p_47=2024&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")
#driver.add_cookie({'name' : 'csm-hit','value':'tb:s-R614YZC0MNEE8MA4GS26|1723461833540&t:1723461833540&adb:adblk_no'})
#driver.add_cookie({'name' : 'lc-main','value':'en_US'})
#
#try:
#    button = driver.find_element(By.ID, 'nav-global-location-popover-link')
#    button.click()
#    time.sleep(3)
#    input = '10001'
#    search = driver.find_element(By.ID, "GLUXZipUpdateInput")
#    search.send_keys(input)
#    time.sleep(3)
#    button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
#    button.click()
#    time.sleep(3)
#    button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
#    button[1].click()
#    time.sleep(3)
#except IndexError:
#    pass
#pagelen = driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-disabled"]').text
#soup = BeautifulSoup(driver.page_source, 'html.parser')
#rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
#for row in rows:
#    data.append(getrowdata(row))
#time.sleep(5)
#df = pd.read_csv('./harpercollins_date_during.csv')
#df['publishDate_'] = df['publishDate'].apply(timeparser)

#df = pd.DataFrame(columns=['title','author','publishDate','mainVersion','mainPrice','mainOriginalPrice','subVersion','subPrice','subOriginalPrice','publishDate_'])
df = pd.read_csv('./harpercollins_date_during.csv')
stucked = 0

for rqyear in range(1750,1900):
    data = []
    driver = webdriver.Chrome( options=chrome_options)
    #driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=41&Adv-Srch-Books-Submit.y=16&p_45={reqdate.month}&p_46=before&p_47={reqdate.year}&qid=1723495742&unfiltered=1&ref=sr_pg_2")
    driver.get(f"https://www.amazon.com/advanced-search/books")
    driver.find_element(By.NAME,'field-publisher').send_keys('harpercollins')
    time.sleep(2)
    sel = driver.find_element(By.NAME,'field-p_n_condition-type')
    sel = select.Select(sel)
    sel.select_by_index(1)
    time.sleep(3)
    sel = driver.find_element(By.NAME,'sort')
    sel = select.Select(sel)
    sel.select_by_value("daterank")
    time.sleep(6)
    sel = driver.find_element(By.NAME,'p_46')
    sel = select.Select(sel)
    sel.select_by_index(1)
    time.sleep(3)
    sel = driver.find_element(By.NAME,'p_45')
    sel = select.Select(sel)
    sel.select_by_index(0)
    time.sleep(3)
    driver.find_element(By.NAME,'p_47').send_keys(rqyear)
    time.sleep(2)
    sel = driver.find_element(By.NAME,"p_n_feature_browse-bin")
    sel = select.Select(sel)
    sel.select_by_index(1)
    time.sleep(1)
    try:
        driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
        time.sleep(3)
        driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
        time.sleep(2)
    except:
        pass

    while BeautifulSoup(driver.page_source, 'html.parser').title.text == 'Sorry! Something went wrong!':
        driver.refresh()
        if BeautifulSoup(driver.page_source, 'html.parser').title.text != 'Sorry! Something went wrong!':
            break
        stucked += 1
        driver.quit()
        time.sleep(stucked*50)
        driver.get(f"https://www.amazon.com/advanced-search/books")
        driver.find_element(By.NAME,'field-publisher').send_keys('harpercollins')
        time.sleep(2)
        sel = driver.find_element(By.NAME,'field-p_n_condition-type')
        sel = select.Select(sel)
        sel.select_by_index(1)
        time.sleep(3)
        sel = driver.find_element(By.NAME,'sort')
        sel = select.Select(sel)
        sel.select_by_value("daterank")
        time.sleep(6)
        sel = driver.find_element(By.NAME,'p_46')
        sel = select.Select(sel)
        sel.select_by_index(1)
        time.sleep(3)
        sel = driver.find_element(By.NAME,'p_45')
        sel = select.Select(sel)
        sel.select_by_index(0)
        time.sleep(3)
        driver.find_element(By.NAME,'p_47').send_keys(rqyear)
        time.sleep(2)
        sel = driver.find_element(By.NAME,"p_n_feature_browse-bin")
        sel = select.Select(sel)
        sel.select_by_index(1)
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
        time.sleep(3)
        driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
    try:
        button = driver.find_element(By.ID, 'nav-global-location-popover-link')
        button.click()
        time.sleep(3)
        input = '10001'
        search = driver.find_element(By.ID, "GLUXZipUpdateInput")
        search.send_keys(input)
        time.sleep(3)
        button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
        button.click()
        time.sleep(3)
        button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
        button[1].click()
        time.sleep(3)
    except IndexError:
        print('indexerror')
        pass
    except:
        pass

    try:
        pagelen = driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-disabled"]').text
        pagelen = int(pagelen)
    except:
        pagelen=1



    for i in range(pagelen):
        #driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins&s=daterank&Adv-Srch-Books-Submit.x=41&Adv-Srch-Books-Submit.y=16&p_45={reqdate.month}&p_46=before&p_47={reqdate.year}&unfiltered=1&ref=sr_adv_b&page={i+1}")
        print(f'processing {rqyear} page {i+1}',end = '\r')
        if random.randint(0,1) == 1:
            driver.refresh()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        while soup.title.text == 'Sorry! Something went wrong!':
            driver.refresh()
            if soup.title.text != 'Sorry! Something went wrong!':
                break
            stucked += 1
            print('waiting',end = '\r')
            time.sleep(stucked*60 if stucked<5 else 180)
            print('processing',end = '\r')
            driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i}&Adv-Srch-Books-Submit.x=32&Adv-Srch-Books-Submit.y=13&p_46=During&p_47={rqyear}&unfiltered=1&ref=sr_adv_b")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            if stucked > 5:
                driver.quit()
                driver = webdriver.Chrome( options=chrome_options)
                driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i}&Adv-Srch-Books-Submit.x=32&Adv-Srch-Books-Submit.y=13&p_46=During&p_47={rqyear}&unfiltered=1&ref=sr_adv_b")
                try:
                    button = driver.find_element(By.ID, 'nav-global-location-popover-link')
                    button.click()
                    time.sleep(3)
                    input = '10001'
                    search = driver.find_element(By.ID, "GLUXZipUpdateInput")
                    search.send_keys(input)
                    time.sleep(3)
                    button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
                    button.click()
                    time.sleep(3)
                    button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
                    button[1].click()
                    time.sleep(3)
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
                    time.sleep(3)
                    driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
                except:
                    pass
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(3)
        rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
        for row in rows:
            data.append(getrowdata(row))
        time.sleep(random.randint(6,10))
        try:
            if driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text != str(pagelen):
                try:
                    driver.find_element(By.CSS_SELECTOR,"a[class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'").click()
                except NoSuchElementException:
                    pass
            elif driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text == str(pagelen):
                break
        except NoSuchElementException:
            pass
    dftemp = pd.DataFrame(data)
    print(dftemp.head,end = '\r')
    try:
        dftemp['publishDate_'] = dftemp['publishDate'].apply(timeparser)
        df = pd.concat([df, dftemp], ignore_index=True)
    except:
        print(dftemp)
        df = df
    



    df.to_csv(f"./data/hapercollins_date_from1750_{rqyear}.csv")
    df.to_csv("./harpercollins_date_during_1600.csv")






rqyear = 1750
data = []
driver = webdriver.Chrome( options=chrome_options)
driver.get(f"https://www.amazon.com/advanced-search/books")
driver.find_element(By.NAME,'field-publisher').send_keys('harpercollins')
time.sleep(2)
sel = driver.find_element(By.NAME,'field-p_n_condition-type')
sel = select.Select(sel)
sel.select_by_index(1)
time.sleep(3)
sel = driver.find_element(By.NAME,'sort')
sel = select.Select(sel)
sel.select_by_value("daterank")
time.sleep(6)
sel = driver.find_element(By.NAME,'p_46')
sel = select.Select(sel)
sel.select_by_value("before")
time.sleep(3)
sel = driver.find_element(By.NAME,'p_45')
sel = select.Select(sel)
sel.select_by_index(0)
time.sleep(3)
driver.find_element(By.NAME,'p_47').send_keys(rqyear)
time.sleep(2)
sel = driver.find_element(By.NAME,"p_n_feature_browse-bin")
sel = select.Select(sel)
sel.select_by_index(1)
time.sleep(1)
try:
    pagelen = driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-disabled"]').text
except:
    pagelen = 0
for i in range(pagelen):
    #driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins&s=daterank&Adv-Srch-Books-Submit.x=41&Adv-Srch-Books-Submit.y=16&p_45={reqdate.month}&p_46=before&p_47={reqdate.year}&unfiltered=1&ref=sr_adv_b&page={i+1}")
    print(f'processing {rqyear} page {i+1}',end = '\r')
    if random.randint(0,1) == 1:
        driver.refresh()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    while soup.title.text == 'Sorry! Something went wrong!':
        driver.refresh()
        if soup.title.text != 'Sorry! Something went wrong!':
            break
        stucked += 1
        print('waiting',end = '\r')
        time.sleep(stucked*60 if stucked<5 else 180)
        print('processing',end = '\r')
        driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i}&Adv-Srch-Books-Submit.x=32&Adv-Srch-Books-Submit.y=13&p_46=During&p_47={rqyear}&unfiltered=1&ref=sr_adv_b")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        if stucked > 5:
            driver.quit()
            driver = webdriver.Chrome( options=chrome_options)
            driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={i}&Adv-Srch-Books-Submit.x=32&Adv-Srch-Books-Submit.y=13&p_46=During&p_47={rqyear}&unfiltered=1&ref=sr_adv_b")
            try:
                button = driver.find_element(By.ID, 'nav-global-location-popover-link')
                button.click()
                time.sleep(3)
                input = '10001'
                search = driver.find_element(By.ID, "GLUXZipUpdateInput")
                search.send_keys(input)
                time.sleep(3)
                button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
                button.click()
                time.sleep(3)
                button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
                button[1].click()
                time.sleep(3)
                time.sleep(2)
                driver.find_element(By.CSS_SELECTOR,'input[name="Adv-Srch-Books-Submit"]').click()
                time.sleep(3)
                driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()
                time.sleep(2)
                driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()
            except:
                pass
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(3)
    rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
    for row in rows:
        data.append(getrowdata(row))
    time.sleep(random.randint(6,10))
    try:
        if driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text != str(pagelen):
            try:
                driver.find_element(By.CSS_SELECTOR,"a[class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'").click()
            except NoSuchElementException:
                pass
        elif driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text == str(pagelen):
            break
    except NoSuchElementException:
        pass
dftemp = pd.DataFrame(data)
print(data[1:30])
print(dftemp.head)
try:
    dftemp['publishDate_'] = dftemp['publishDate'].apply(timeparser)
    df = pd.concat([df, dftemp], ignore_index=True)
except:
    print(dftemp)
    df = df




df.to_csv(f"./data/hapercollins_date_from1600_{rqyear}.csv")
df.to_csv("./harpercollins_date_during_1600.csv")

<bound method NDFrame.head of                                                title    author  publishDate  \
0                                               None      None         None   
1  Planet Earth/Inside Out by Gail Gibbons (1998-...      None  Jan 1, 1750   
2  The King's English (Penguin Modern Classics) b...      None  Jan 1, 1750   
3  Coming to America: History of Immigration and ...      None  Jan 1, 1750   
4  Mittens (My First I Can Read) by Lola M. Schae...  Audio CD  Jan 1, 1750   
5         Time Train by Paul Fleischman (1994-02-28)      None  Jan 1, 1750   
6  Nasty, Stinky Sneakers by Eve Bunting (1995-04...      None  Jan 1, 1750   

  mainVersion mainPrice mainOriginalPrice                 subVersion subPrice  \
0        None      None              None                       None     None   
1  Paperback       None              None           Library Binding      None   
2  Paperback     $39.69              None                    Kindle    $11.99   
3  Paperback 

In [347]:

driver = webdriver.Chrome( options=chrome_options)
driver.get("https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45=8&p_46=before&p_47=2024&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")
driver.add_cookie({'name' : 'csm-hit','value':'tb:s-R614YZC0MNEE8MA4GS26|1723461833540&t:1723461833540&adb:adblk_no'})
driver.add_cookie({'name' : 'lc-main','value':'en_US'})
driver.get("https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=2866&Adv-Srch-Books-Submit.y=85&p_45=10&p_46=before&p_47=1700&unfiltered=1&ref=sr_adv_b")
button = driver.find_element(By.ID, 'nav-global-location-popover-link')
button.click()

input = '10001'
search = driver.find_element(By.ID, "GLUXZipUpdateInput")
search.send_keys(input)
time.sleep(3)
button = driver.find_element(By.CSS_SELECTOR,'input[aria-labelledby="GLUXZipUpdate-announce"]')
button.click()
time.sleep(3)
button = driver.find_elements(By.CSS_SELECTOR,'input[aria-labelledby="GLUXConfirmClose-announce"]')
button[1].click()
time.sleep(3)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="nav-global-location-popover-link"]"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001047a8848 chromedriver + 5179464
1   chromedriver                        0x00000001047a027a chromedriver + 5145210
2   chromedriver                        0x00000001043172b0 chromedriver + 389808
3   chromedriver                        0x00000001043633e1 chromedriver + 701409
4   chromedriver                        0x0000000104363691 chromedriver + 702097
5   chromedriver                        0x00000001043a5464 chromedriver + 971876
6   chromedriver                        0x00000001043857dd chromedriver + 841693
7   chromedriver                        0x00000001043a2b9b chromedriver + 9614

In [348]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [316]:
rows = soup.find_all('div', class_='a-section a-spacing-small a-spacing-top-small')
row = rows[1]
print(row.find('div', class_="a-row a-spacing-mini").find_all('div', class_="a-row a-size-base a-color-base")[1].find('span', class_="a-offscreen").text)

$9.99


In [346]:
driver.get("https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&Adv-Srch-Books-Submit.x=2945&Adv-Srch-Books-Submit.y=512&p_45=1&p_46=before&p_47=2018&unfiltered=1&ref=sr_adv_b")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
0   chromedriver                        0x0000000108c3a848 chromedriver + 5179464
1   chromedriver                        0x0000000108c3227a chromedriver + 5145210
2   chromedriver                        0x00000001087a92b0 chromedriver + 389808
3   chromedriver                        0x0000000108780614 chromedriver + 222740
4   chromedriver                        0x000000010881f8ed chromedriver + 874733
5   chromedriver                        0x00000001088344a9 chromedriver + 959657
6   chromedriver                        0x0000000108817553 chromedriver + 841043
7   chromedriver                        0x00000001087e87f6 chromedriver + 649206
8   chromedriver                        0x00000001087e905e chromedriver + 651358
9   chromedriver                        0x0000000108bfdab0 chromedriver + 4930224
10  chromedriver   

In [33]:
temp = 'Sep 6, 2019'
print(timeparser(temp))

None


2019-09-06


In [351]:
BeautifulSoup(driver.page_source, 'html.parser').title.text == 'Sorry! Something went wrong!'

True

In [361]:
driver = webdriver.Chrome( options=chrome_options)

In [366]:
driver.get(f"https://www.amazon.com/s?i=stripbooks&rh=p_30%3Aharpercollins%2Cp_n_condition-type%3A1294423011%2Cp_n_feature_browse-bin%3A2656022011&s=daterank&page={10}&Adv-Srch-Books-Submit.x=50&Adv-Srch-Books-Submit.y=9&p_45={reqdate.month-1}&p_46=before&p_47={reqdate.year}&qid=1723456851&refresh=4&unfiltered=1&ref=sr_pg_1")

In [367]:
driver.refresh()

In [10]:
sel = driver.find_element(By.NAME,'sort')
select = select.Select(sel)
select.select_by_index(5)

In [19]:
driver = webdriver.Chrome( options=chrome_options)
driver.get("https://www.amazon.com/s?k=sex&i=stripbooks&crid=11695MSDSX52V&sprefix=se%2Cstripbooks%2C452&ref=nb_sb_noss_2")

In [15]:
driver.find_element(By.CSS_SELECTOR,"a[class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'").click()

In [24]:
driver.find_element(By.CSS_SELECTOR,'span[class="a-button-text a-declarative"]').click()

In [25]:
driver.find_element(By.CSS_SELECTOR,'a[id="s-result-sort-select_4"]').click()

In [29]:
driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text

'75'

In [33]:
driver.find_element(By.CSS_SELECTOR,'span[class="s-pagination-item s-pagination-selected"]').text == str(pagelen)

True